In [ ]:
from core import initialize_bodies
from 
import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import animation

In [1]:
import numpy as np
from core import initialize_bodies
from quadtree import Node
from quadtree import Quadtree

# initialize bodies with random masses, positions, and velocities over set range
# bodies[n] = Body(mass, position [x, y], velocity [vx, vy])
bodies = initialize_bodies(5, mass_range = [1, 10], position_range = [-25, 25], velocity_range = [-5, 5])
positions = [body.position for body in bodies]
print(positions)
# printing sample values
# print(bodies[0].mass)
# print(bodies[0].position)
# print(bodies[0].velocity)
# print(len(bodies))

# last resort below. Had to format it like this bc I had no idea how to utilize it otherwise. feel free to make it work with the class itself
# formatbodies = [[body.mass for body in bodies], [[body.position[0] for body in bodies], [body.position[1] for body in bodies]], [body.velocity for body in bodies]]

# print([body.position for body in bodies])

QuadTest = Quadtree(bodies, 50)
print(Quadtest.nodelist)

[array([-0.26578956, -0.34114584]), array([17.20780148, 23.04132874]), array([-19.36970751, -15.94282974]), array([-11.88344185,  -4.19934189]), array([10.30865295, 21.23743464])]
subdivide bodies is[<core.Body object at 0x000002101AE34B50>, <core.Body object at 0x000002101AE55AD0>, <core.Body object at 0x000002101A868A10>, <core.Body object at 0x000002104957D510>, <core.Body object at 0x000002104957EC50>]
points in node[0, 1, 2, 3, 4]
points in node[0, 1, 2, 3, 4]
points in node[0, 1, 2, 3, 4]
points in node[0, 1, 2, 3, 4]
points in node[0, 1, 2, 3, 4]
quad1bodies[<core.Body object at 0x000002101AE55AD0>, <core.Body object at 0x000002104957EC50>]
quad2bodies[]
quad3bodies[]
quad4bodies[<core.Body object at 0x000002101AE34B50>, <core.Body object at 0x000002101A868A10>, <core.Body object at 0x000002104957D510>]
new node quad 1<quadtree.Node object at 0x000002101AFC2A10>
new node quad 2<quadtree.Node object at 0x000002104957D910>
new node quad 3<quadtree.Node object at 0x000002104957DCD0

AttributeError: 'int' object has no attribute 'position'

In [ ]:
# Example animation using matplotlib

%matplotlib widget

# Time step for position updates
dt = 0.1

# Set up the plot
fig, ax = plt.subplots()
ax.set_xlim(-30, 30) # X-axis limits
ax.set_ylim(-30, 30) # Y-axis limits
ax.set_title("Particle Simulation", color="black")  # Title
ax.set_xlabel("X Position", color="black")  # X-axis label
ax.set_ylabel("Y Position", color="black")  # Y-axis label
ax.tick_params(axis='x', colors="black")  # X-axis ticks
ax.tick_params(axis='y', colors="black")  # Y-axis ticks
ax.set_facecolor("black")  # Set background color
fig.patch.set_facecolor("lightgray")  # Outside the plot

# Create scatter plot with initial positions
scat = ax.scatter(
    [body.position[0] for body in bodies], # Initial x-coordinate of bodies
    [body.position[1] for body in bodies], # Initial y-coordinate of bodies
    s=[body.mass * 10 for body in bodies], # Size of bodies proportional to mass
    c="yellow",  # Color of bodies
)

# Update function for animation
def update_position(frame):

    updated_positions = [] # Initialize as a list

    for body in bodies:
        body.position += body.velocity * dt  # Update positions
        updated_positions.append(body.position)  # Collect new positions
    scat.set_offsets(updated_positions)  # Update scatter plot
    
    return scat

# Create the animation
ani = animation.FuncAnimation(
    fig=fig,
    func=update_position,
    frames=range(50),  # Number of frames
    interval=50,  # Time between frames in milliseconds
    repeat=False,  # Stop after the last frame
)

# Show the plot
plt.show()

In [ ]:
# import numpy as np
# from quadtree import Node
# from quadtree import Quadtree



# # Testing Quadtree
